In [1]:
import sys, os, pickle, commands, re
from glob import glob
sys.path.append('../lib')

In [2]:
from analysis_utilities import drawOnCMSCanvas, DSetLoader

Welcome to JupyROOT 6.12/07


In [3]:
procOrder = [
    'mu', 'tau',
    'DstPip', 'DstPi0', 'DstPipPim', 'DstPipPi0', 'DstPi0Pi0',
    'TauDstPip', 'TauDstPi0',
    'DstmD0', 'DstmDp', 'DstmDsp', 'BmDstmHc', 'BpDstmHc', 'antiB0DstmHc', 
]

In [4]:
separationLine = [1,6,8,14]

In [5]:
MCsample = {
'tau' : DSetLoader('B0_TauNuDmst_PUc0'),
'mu' : DSetLoader('B0_MuNuDmst_PUc0'),
# 'mu' : DSetLoader('B0_MuNuDmst_SoftQCDall_PUc0'),
'DstmD0' : DSetLoader('B0_DstmD0_PUc0'),
'DstmDp' : DSetLoader('B0_DstmDp_PUc0'),
'DstmDsp' : DSetLoader('B0_DstmDsp_PUc0'),
'BpDstmHc' : DSetLoader('Bp_DstmHc_PUc0'),
'BmDstmHc' : DSetLoader('Bm_DstmHc_PUc0'),
'antiB0DstmHc' : DSetLoader('antiB0_DstmHc_PUc0'),
'DstPip' : DSetLoader('Bp_MuNuDstst_Pip_PUc0'),
'DstPi0' : DSetLoader('B0_MuNuDstst_Pi0_PUc0'),
'DstPipPi0' : DSetLoader('Bp_MuNuDstst_PipPi0_PUc0'),
'DstPipPim' : DSetLoader('B0_MuNuDstst_PipPim_PUc0'),
# 'DstPipPi0' : DSetLoader('Bp_MuNuDstPipPi0_PUc0'),
# 'DstPipPim' : DSetLoader('B0_MuNuDstPipPim_PUc0'),
'DstPi0Pi0' : DSetLoader('B0_MuNuDstst_Pi0Pi0_PUc0'),
'TauDstPi0' : DSetLoader('B0_TauNuDstst_Pi0_PUc0'),
'TauDstPip' : DSetLoader('Bp_TauNuDstst_Pip_PUc0')
}

In [6]:
labelTLatex = {
             'data'     : 'Data',
             'mu'       : 'B^{0}#rightarrow D*#mu^{+}#nu',
             'tau'      : 'B^{0}#rightarrow D*#tau^{+}#nu',
             
             'DstPi0': 'B^{0}#rightarrow D**(D*#pi^{0})#mu#nu',
             'DstPip': 'B^{+}#rightarrow D**(D*#pi^{+})#mu#nu',
             'DstPipPim': 'B^{0}#rightarrow D**(D*#pi^{+}#pi^{-})#mu#nu',
             'DstPi0Pi0': 'B^{0}#rightarrow D**(D*#pi^{0}#pi^{0})#mu#nu',
             'DstPipPi0': 'B^{+}#rightarrow D**(D*#pi^{+}#pi^{0})#mu#nu',
             
             'TauDstPi0' : 'B^{0}#rightarrow D**(D*#pi^{0})#tau#nu',
             'TauDstPip' : 'B^{+}#rightarrow D**(D*#pi^{+})#tau#nu',
             
             'DstmD0'   : 'B^{0}#rightarrow D*D^{0}X',
             'DstmDp'   : 'B^{0}#rightarrow D*D^{+}X',
             'DstmDsp'   : 'B^{0}#rightarrow D*D_{s}^{+}X',
             'BmDstmHc' : 'B^{-}#rightarrow D*H_{c}(#muX)',
             'BpDstmHc' : 'B^{+}#rightarrow D*H_{c}(#muX)',
             'antiB0DstmHc' : '#bar{B}^{0}#rightarrow D*H_{c}(#muX)',
            }

In [12]:
labelLatex = {
             'data'         : 'Data',
             'mu'           : r'$B^0 \to D^{*-}\mu\nu$',
             'tau'          : r'$B^0 \to D^{*-}\tau\nu$',
             
             'DstPi0'       : r'$B^+ \to D^{*-}\pi^{+}\mu\nu$',
             'DstPip'       : r'$B^0 \to D^{*-}\pi^0\mu\nu$',
             'DstPipPim'    : r'$B^0 \to D^{*-}\pi^{+}\pi^{-}\mu\nu$',
             'DstPi0Pi0'    : r'$B^+ \to D^{*-}\pi^{+}\pi^{0}\mu\nu$',
             'DstPipPi0'    : r'$B^0 \to D^{*-}\pi^{0}\pi^{0}\mu\nu$',
             
             'TauDstPi0'    : r'$B^+ \to D^{*-}\pi^{+}\tau\nu$',
             'TauDstPip'    : r'$B^0 \to D^{*-}\pi^0\tau\nu$',
             
             'DstmDsp'      : r'$B^0\to D^{*-}D_s$',
             'DstmD0'       : r'$B^0\to D^{*-}D^0X$',
             'DstmDp'       : r'$B^0\to D^{*-}D^+X$',
             'BpDstmHc'     : r'$B^+\to D^{*-}H_c$',
             'BmDstmHc'     : r'$B^-\to D^{*-}H_c$',
             'antiB0DstmHc' : r'$\overline{B}^0\to D^{*-}H_c$'
            }

# N-tuplizer table

In [13]:
latexTable = r'''
\begin{tabular}{|c|c|c|c|}
 \hline
 Process & $\varepsilon_\text{trg}$ [\%]& $\varepsilon_\text{cand}$ [\%]& $\varepsilon_\text{ntp}$ [\%] \\
 \hline
 \hline
'''
for i, p in enumerate(procOrder):      
    eff = MCsample[p].effCand
    
    latexTable += ' ' + labelLatex[p]
    latexTable += r' & '
    latexTable += r'${:2.2f} \pm {:2.2f}$'.format(100*eff['effCAND_trg'][0], 100*eff['effCAND_trg'][1])
    latexTable += r' & '
    latexTable += r'${:2.2f} \pm {:2.2f}$'.format(100*eff['effCAND_cand'][0], 100*eff['effCAND_cand'][1])
    latexTable += r' & '
    latexTable += r'${:2.2f} \pm {:2.2f}$'.format(100*eff['effCAND'][0], 100*eff['effCAND'][1])
    latexTable += r' \\'
    latexTable += '\n'
    
    if i in separationLine:
        latexTable += ' \\hline\n'
        
latexTable += r'\end{tabular}' + '\n'
print latexTable


\begin{tabular}{|c|c|c|c|}
 \hline
 Process & $\varepsilon_\text{trg}$ [\%]& $\varepsilon_\text{cand}$ [\%]& $\varepsilon_\text{ntp}$ [\%] \\
 \hline
 \hline
 $B^0 \to D^{*-}\mu\nu$ & $36.54 \pm 0.00$ & $10.14 \pm 0.00$ & $3.70 \pm 0.00$ \\
 $B^0 \to D^{*-}\tau\nu$ & $40.44 \pm 0.02$ & $12.90 \pm 0.02$ & $5.22 \pm 0.01$ \\
 \hline
 $B^0 \to D^{*-}\pi^0\mu\nu$ & $40.90 \pm 0.02$ & $16.94 \pm 0.02$ & $6.93 \pm 0.01$ \\
 $B^+ \to D^{*-}\pi^{+}\mu\nu$ & $40.86 \pm 0.03$ & $15.85 \pm 0.03$ & $6.48 \pm 0.01$ \\
 $B^0 \to D^{*-}\pi^{+}\pi^{-}\mu\nu$ & $39.65 \pm 0.01$ & $21.84 \pm 0.02$ & $8.66 \pm 0.01$ \\
 $B^0 \to D^{*-}\pi^{0}\pi^{0}\mu\nu$ & $39.78 \pm 0.02$ & $15.41 \pm 0.02$ & $6.13 \pm 0.01$ \\
 $B^+ \to D^{*-}\pi^{+}\pi^{0}\mu\nu$ & $39.68 \pm 0.02$ & $14.64 \pm 0.02$ & $5.81 \pm 0.01$ \\
 \hline
 $B^0 \to D^{*-}\pi^0\tau\nu$ & $43.62 \pm 0.04$ & $16.30 \pm 0.05$ & $7.11 \pm 0.02$ \\
 $B^+ \to D^{*-}\pi^{+}\tau\nu$ & $42.77 \pm 0.04$ & $15.49 \pm 0.04$ & $6.62 \pm 0.02$ \\
 \hline
 

# Category effieincy

In [17]:
MCsample['mu'].eff

sMC.getSkimEff(category.name+'_'+mcType)

In [19]:
latexTable = r'''
\begin{tabular}{|c|c|c|c|}
 \hline
 & Low & Mid & High \\
 \hline
 \hline
'''
for i, p in enumerate(procOrder):    
    latexTable += ' ' + labelLatex[p]
    
    for category in ['Low', 'Mid', 'High']:
        eff = MCsample[p].getSkimEff(category+'_bare')
        latexTable += r' & '
        latexTable += r'${:2.2f} \pm {:2.2f}$'.format(100*eff[0], 100*eff[1])
    latexTable += r' \\'
    latexTable += '\n'
    
    if i in separationLine:
        latexTable += ' \\hline\n'
        
latexTable += r'\end{tabular}' + '\n'
print latexTable


\begin{tabular}{|c|c|c|c|}
 \hline
 & Low & Mid & High \\
 \hline
 \hline
 $B^0 \to D^{*-}\mu\nu$ & $15.71 \pm 0.02$ & $12.10 \pm 0.02$ & $13.50 \pm 0.02$ \\
 $B^0 \to D^{*-}\tau\nu$ & $21.94 \pm 0.08$ & $13.48 \pm 0.07$ & $11.04 \pm 0.06$ \\
 \hline
 $B^0 \to D^{*-}\pi^0\mu\nu$ & $8.11 \pm 0.04$ & $5.23 \pm 0.03$ & $4.09 \pm 0.03$ \\
 $B^+ \to D^{*-}\pi^{+}\mu\nu$ & $21.52 \pm 0.09$ & $15.48 \pm 0.08$ & $15.47 \pm 0.08$ \\
 $B^0 \to D^{*-}\pi^{+}\pi^{-}\mu\nu$ & $2.02 \pm 0.01$ & $1.17 \pm 0.01$ & $0.72 \pm 0.01$ \\
 $B^0 \to D^{*-}\pi^{0}\pi^{0}\mu\nu$ & $7.75 \pm 0.04$ & $4.99 \pm 0.04$ & $3.78 \pm 0.03$ \\
 $B^+ \to D^{*-}\pi^{+}\pi^{0}\mu\nu$ & $22.21 \pm 0.07$ & $16.05 \pm 0.06$ & $15.59 \pm 0.06$ \\
 \hline
 $B^0 \to D^{*-}\pi^0\tau\nu$ & $8.15 \pm 0.09$ & $4.22 \pm 0.06$ & $2.58 \pm 0.05$ \\
 $B^+ \to D^{*-}\pi^{+}\tau\nu$ & $22.83 \pm 0.13$ & $13.63 \pm 0.11$ & $10.13 \pm 0.09$ \\
 \hline
 $B^0\to D^{*-}D^0X$ & $1.85 \pm 0.05$ & $1.01 \pm 0.04$ & $0.61 \pm 0.03$ \\
 $B^0\to D